## Create features for each report

In [1]:
import pandas as pd
import numpy as np
import os
import spacy
from transformers import pipeline
import re
from transformers import AutoTokenizer

from numpy.linalg import norm

import torch
from numpy.linalg import norm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification

In [2]:
df = pd.read_csv('combined_train.csv')
df

,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,TICKER,SECTOR,MOMENTUM,VALUE,SIZE,BETA,SEDOL,ISIN,RETURNS,report_type
0,f_4csLsjAhudR2mcDosUbKD5,LULULEMON ATHLETICA INC,US5500211090,2024-03-21,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,2024-03,2024-03-28,55002110,...,LULU,Consumer Discretionary,44.9034,-0.609,10.840906,1.217020,B23FN39,US5500211090,-0.163652,Annual
1,f_9tgeLS94nO4peRbBEIbvje,LULULEMON ATHLETICA INC,US5500211090,2024-08-29,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,2024-08,2024-08-30,55002110,...,LULU,Consumer Discretionary,-32.0468,-0.131,10.408057,0.576961,B23FN39,US5500211090,0.003132,Quarterly
2,f_PDplDokn,LULULEMON ATHLETICA INC,US5500211090,2023-08-31,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,2023-08,2023-08-31,55002110,...,LULU,Consumer Discretionary,26.1935,-0.758,10.815705,1.478690,B23FN39,US5500211090,0.007212,Quarterly
3,f_WoQLjVEq,LULULEMON ATHLETICA INC,US5500211090,2023-12-07,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,2023-12,2023-12-29,55002110,...,LULU,Consumer Discretionary,38.3593,-0.938,11.110417,1.365380,B23FN39,US5500211090,0.144338,Quarterly
4,f_cC5GK9NDnFMWH7KTPOIlox,LULULEMON ATHLETICA INC,US5500211090,2024-12-05,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,2024-12,2024-12-31,55002110,...,LULU,Consumer Discretionary,-37.2841,-0.506,10.794169,-0.378504,B23FN39,US5500211090,0.192572,Quarterly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,o_f9saaXvCQe9wIyNvILHS3l,MOODY'S CORP,US6153691059,2023-07-06,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,2023-07,2023-07-31,61536910,...,MCO,Financials,13.1608,-0.850,11.078349,1.627940,2252058,US6153691059,0.014466,Event/Other
4242,o_fwe9aJYEVTFtxduTAdK6sd,GE VERNOVA INC,US36828A1016,2024-12-10,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,data/reports/GE_VERNOVA_INC/o_fwe9aJYEVTFtxduT...,2024-12,2024-12-31,36828A10,...,GEV,Industrials,NaN,-0.803,11.419023,0.218221,BP6H4Y1,US36828A1016,-0.014815,Event/Other
4243,o_gAqntEyHWJXJDmnXkTAQEe,BANK OF AMERICA CORP,US0605051046,2024-09-30,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,2024-09,2024-09-30,06050510,...,BAC,Financials,51.9570,2.520,12.630449,1.355190,2295677,US0605051046,-0.019726,Earnings
4244,o_gdd49aPyFdkFEH34pnybsJ,CITIGROUP INC,US1729674242,2024-06-18,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,data/reports/CITIGROUP_INC/o_gdd49aPyFdkFEH34p...,2024-06,2024-06-28,17296742,...,C,Financials,40.8893,3.993,11.673367,1.276030,2297907,US1729674242,0.018456,Event/Other


In [3]:
# Load embedding tracker list
with open("embedding_file.txt") as f:
    embedding_files = set(x.strip() for x in f if x.strip())

def get_emb_path(path):
    if path in embedding_files:
        return path.replace(".txt", "_emb.parquet")
    return None

# Use pages_path (since that's what you used in embedding loop)
df["embeddings_path"] = df["pages_path"].apply(get_emb_path)

# Check missing
missing_count = df["embeddings_path"].isna().sum()
print(f"Number of paths without embeddings: {missing_count}")

df = df.dropna().reset_index(drop=True)
df

Number of paths without embeddings: 34


,report_id,company_name,isin,reported_at,pdf_path,pages_path,blocks_path,year_month,DATE,SYMBOL,...,SECTOR,MOMENTUM,VALUE,SIZE,BETA,SEDOL,ISIN,RETURNS,report_type,embeddings_path
0,f_4csLsjAhudR2mcDosUbKD5,LULULEMON ATHLETICA INC,US5500211090,2024-03-21,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...,2024-03,2024-03-28,55002110,...,Consumer Discretionary,44.9034,-0.609,10.840906,1.217020,B23FN39,US5500211090,-0.163652,Annual,data/reports/LULULEMON_ATHLETICA_INC/f_4csLsjA...
1,f_9tgeLS94nO4peRbBEIbvje,LULULEMON ATHLETICA INC,US5500211090,2024-08-29,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...,2024-08,2024-08-30,55002110,...,Consumer Discretionary,-32.0468,-0.131,10.408057,0.576961,B23FN39,US5500211090,0.003132,Quarterly,data/reports/LULULEMON_ATHLETICA_INC/f_9tgeLS9...
2,f_PDplDokn,LULULEMON ATHLETICA INC,US5500211090,2023-08-31,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...,2023-08,2023-08-31,55002110,...,Consumer Discretionary,26.1935,-0.758,10.815705,1.478690,B23FN39,US5500211090,0.007212,Quarterly,data/reports/LULULEMON_ATHLETICA_INC/f_PDplDok...
3,f_WoQLjVEq,LULULEMON ATHLETICA INC,US5500211090,2023-12-07,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...,2023-12,2023-12-29,55002110,...,Consumer Discretionary,38.3593,-0.938,11.110417,1.365380,B23FN39,US5500211090,0.144338,Quarterly,data/reports/LULULEMON_ATHLETICA_INC/f_WoQLjVE...
4,f_cC5GK9NDnFMWH7KTPOIlox,LULULEMON ATHLETICA INC,US5500211090,2024-12-05,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...,2024-12,2024-12-31,55002110,...,Consumer Discretionary,-37.2841,-0.506,10.794169,-0.378504,B23FN39,US5500211090,0.192572,Quarterly,data/reports/LULULEMON_ATHLETICA_INC/f_cC5GK9N...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4186,o_aeN2BhuJP2gi3M2kxsHkPm,BANK OF AMERICA CORP,US0605051046,2024-06-30,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...,2024-06,2024-06-28,06050510,...,Financials,43.3526,2.150,12.634439,1.420270,2295677,US0605051046,0.000499,Earnings,data/reports/BANK_OF_AMERICA_CORP/o_aeN2BhuJP2...
4187,o_biaNyxVrdz2sRiW2NToHAP,CITIGROUP INC,US1729674242,2024-07-12,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...,2024-07,2024-07-31,17296742,...,Financials,38.8723,4.077,11.741387,1.399230,2297907,US1729674242,0.022376,Earnings,data/reports/CITIGROUP_INC/o_biaNyxVrdz2sRiW2N...
4188,o_f9saaXvCQe9wIyNvILHS3l,MOODY'S CORP,US6153691059,2023-07-06,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...,2023-07,2023-07-31,61536910,...,Financials,13.1608,-0.850,11.078349,1.627940,2252058,US6153691059,0.014466,Event/Other,data/reports/MOODY'S_CORP/o_f9saaXvCQe9wIyNvIL...
4189,o_gAqntEyHWJXJDmnXkTAQEe,BANK OF AMERICA CORP,US0605051046,2024-09-30,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...,2024-09,2024-09-30,06050510,...,Financials,51.9570,2.520,12.630449,1.355190,2295677,US0605051046,-0.019726,Earnings,data/reports/BANK_OF_AMERICA_CORP/o_gAqntEyHWJ...


In [5]:
# Path to the unified parquet
OUT_PATH_ALL = "category_embeddings.parquet"

# Load
cat_df = pd.read_parquet(OUT_PATH_ALL)
cat_df

,domain,category,phrase,embedding
0,Finance,Sales/Revenue,sales,"[0.027946406975388527, 0.015969375148415565, 0..."
1,Finance,Sales/Revenue,revenue,"[0.009392556734383106, 0.0008155024261213839, ..."
2,Finance,Sales/Revenue,turnover,"[-0.027533171698451042, 0.030631572008132935, ..."
3,Finance,Sales/Revenue,top line,"[0.011598790064454079, -0.000995350768789649, ..."
4,Finance,Sales/Revenue,net sales,"[0.01926872693002224, -0.00971765723079443, 0...."
...,...,...,...,...
337,Analyst,Company,guidance provided,"[0.016271786764264107, 0.013869144022464752, 0..."
338,Analyst,Company,board of directors statement,"[0.010265818797051907, -0.022233307361602783, ..."
339,Analyst,Company,chairman’s letter,"[0.05726439878344536, 0.0073680575005710125, 0..."
340,Analyst,Company,quarterly update,"[-0.01161214243620634, 0.030039634555578232, 0..."


In [7]:
# --- Device selection ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Sentiment model: FinBERT ---
finbert_name = "ProsusAI/finbert"
finbert_tokenizer = BertTokenizer.from_pretrained(finbert_name)
finbert_model = BertForSequenceClassification.from_pretrained(finbert_name, num_labels=3)
finbert_model = finbert_model.to(device)

# --- MNLI model for Risk & Analyst ---
mnli_model_name = "facebook/bart-large-mnli"
mnli_pipeline = pipeline("zero-shot-classification", model=mnli_model_name, device=0 if torch.cuda.is_available() else -1)

Device set to use cpu


In [8]:
# --- Spacy for tense detection ---
nlp = spacy.load("en_core_web_sm")

# --- Cosine similarity ---
def cosine_sim(a, b):
    return np.dot(a, b) / (norm(a) * norm(b) + 1e-9)

# --- Domain-specific thresholds ---
DOMAIN_THRESHOLDS = {
    "Finance": 0.35,
    "Business": 0.35
}

# --- Final required columns ---
required_cols = [
    "Sales","Expense","Profit","Ops","Liq","Inv","Fin","Lit","Emp","Tax","Acct",
    "Prod","Buyer","Process","Loc","Promo","Supplier","Mgmt","Strategy",
    "IndStruct","BizDesc","RiskFirm","RiskGeneric","Obj","Subj","Fwd","Hist",
    "analyst","Sentiment"
]

# --- Mapping from cat_df categories → required_cols ---
col_mapping = {
    # Finance
    "Sales/Revenue": "Sales",
    "Expense/Cost": "Expense",
    "Profit/Loss": "Profit",
    "Operations": "Ops",
    "Liquidity": "Liq",
    "Investment": "Inv",
    "Financing": "Fin",
    "Litigation": "Lit",
    "Employment": "Emp",
    "Regulation/Tax": "Tax",
    "Accounting": "Acct",
    # Business
    "Product/Service": "Prod",
    "Buyer/Customer": "Buyer",
    "Process/Technology": "Process",
    "Location/Market": "Loc",
    "Promotion/Marketing": "Promo",
    "Supplier/Partner": "Supplier",
    "Management": "Mgmt",
    "Strategy": "Strategy",
    "Industry/Structure": "IndStruct",
    "Business Description": "BizDesc"
}

In [ ]:
# --- Finance/Business classification with embeddings ---
def classify_with_embeddings(sent_embeds, domain, default_threshold=0.10):
    threshold = DOMAIN_THRESHOLDS.get(domain, default_threshold)
    domain_df = cat_df[cat_df["domain"] == domain]
    categories = domain_df["category"].unique()

    results = []
    for emb in sent_embeds:
        row = {col_mapping.get(cat, cat): 0 for cat in categories}
        for cat in categories:
            cat_embs = domain_df[domain_df["category"] == cat]["embedding"].tolist()
            sims = [cosine_sim(np.array(emb), np.array(c)) for c in cat_embs]
            max_sim = max(sims) if sims else 0
            if max_sim >= threshold:
                row[col_mapping.get(cat, cat)] = 1
        results.append(row)
    return results

# --- Risk classification with MNLI ---
def classify_risk_mnli(sentences):
    candidate_labels = ["Firm-specific Risk", "Generic Risk"]
    results = []
    for s in sentences:
        row = {"RiskFirm": 0, "RiskGeneric": 0}
        res = mnli_pipeline(s, candidate_labels=candidate_labels, multi_label=True)
        for label, score in zip(res["labels"], res["scores"]):
            if label == "Firm-specific Risk" and score > 0.5:
                row["RiskFirm"] = 1
            elif label == "Generic Risk" and score > 0.5:
                row["RiskGeneric"] = 1
        results.append(row)
    return results

# --- Analyst classification with MNLI ---
def classify_analyst_mnli(sentences):
    candidate_labels = ["Analyst", "Company"]
    results = []
    for s in sentences:
        row = {"analyst": 0}
        res = mnli_pipeline(s, candidate_labels=candidate_labels, multi_label=False)
        if res["labels"][0] == "Analyst" and res["scores"][0] > 0.5:
            row["analyst"] = 1
        results.append(row)
    return results

# --- Sentiment classification with FinBERT ---
def classify_sentiment(sentences, max_len=512):
    results = []
    for s in sentences:
        inputs = finbert_tokenizer(
            s, return_tensors="pt", padding=True, truncation=True, max_length=max_len
        ).to(device)
        with torch.no_grad():
            logits = finbert_model(**inputs).logits
        pred = torch.argmax(logits, dim=-1).cpu().item()
        # FinBERT: 0 = negative, 1 = neutral, 2 = positive
        results.append(-1 if pred == 0 else (1 if pred == 2 else 0))
    return results

# --- Subjectivity (rule-based) ---
def classify_subjectivity(sentence: str):
    text = sentence.lower()
    subj_words = ["believe","think","expect","anticipate","may","might",
                  "could","should","possible","likely","unlikely","estimate"]
    if any(w in text for w in subj_words):
        return {"Subj": 1, "Obj": 0}
    if re.search(r"\d", sentence) or "%" in sentence or re.search(r"\b20\d{2}\b", sentence):
        return {"Subj": 0, "Obj": 1}
    return {"Subj": 1, "Obj": 0}

# --- Forward vs Historical (rule-based) ---
def classify_forward_past(sentence):
    doc = nlp(sentence)
    fwd, hist = 0, 0
    for token in doc:
        if token.tag_ == "MD" or token.text.lower() in {"will","shall"}:
            fwd = 1
        elif token.tag_ in {"VBD","VBN"}:
            hist = 1
    return {"Fwd": fwd, "Hist": hist}


# ================== MAIN LOOP ==================
processed = set()
tracker_file = "pages_importance.txt"
if os.path.exists(tracker_file):
    with open(tracker_file) as f:
        processed = set(x.strip() for x in f)

counter = 0
for idx, row in df.iterrows():
    path = row["pages_path"]
    emb_path = row.get("embeddings_path", None)

    if not emb_path or not os.path.exists(emb_path):
        print(f"⚠️ No embeddings for {path}, skipping")
        continue
    if path in processed:
        print(f"Skipping {path}")
        continue

    try:
        # Load file + embeddings
        file_df = pd.read_json(path, lines=True)
        emb_df = pd.read_parquet(emb_path)
        if "sentence" not in file_df or "embedding" not in emb_df:
            print(f"⚠️ Missing data in {path}, skipping")
            continue

        sents = file_df["sentence"].tolist()
        sent_embeds = emb_df["embedding"].tolist()

        # --- Run classifiers ---
        fin_out     = classify_with_embeddings(sent_embeds, "Finance")
        biz_out     = classify_with_embeddings(sent_embeds, "Business")
        risk_out    = classify_risk_mnli(sents)
        analyst_out = classify_analyst_mnli(sents)
        sent_out    = classify_sentiment(sents)

        # --- Heuristics
        subj_out  = [classify_subjectivity(s) for s in sents]
        tense_out = [classify_forward_past(s) for s in sents]

        # --- Merge row-wise
        results = []
        for i in range(len(sents)):
            row_out = {}
            row_out.update(fin_out[i])
            row_out.update(biz_out[i])
            row_out["Sentiment"] = sent_out[i]
            row_out.update(tense_out[i])
            row_out.update(risk_out[i])
            row_out.update(subj_out[i])
            row_out.update(analyst_out[i])
            results.append(row_out)

        results_df = pd.DataFrame(results)

        # --- Normalize to required_cols ---
        for c in required_cols:
            if c not in results_df:
                results_df[c] = 0
        results_df = results_df[required_cols]

        # --- Drop old cols and concat ---
        file_df = file_df.drop(columns=[c for c in required_cols if c in file_df], errors="ignore")
        file_df = pd.concat([file_df.reset_index(drop=True), results_df.reset_index(drop=True)], axis=1)

        # Save cleaned file
        file_df.to_json(path, orient="records", lines=True)

        with open(tracker_file, "a") as f:
            f.write(path + "\n")

        counter += 1
        print(f"{counter}. Processed {path}", flush=True)

    except Exception as e:
        print(f"❌ Error processing {path}: {e}", flush=True)